## Status Invest

This script reads all stocks data available in https://statusinvest.com.br/ using scrapers.\
Giving you the possibility to filter out only the relevant stocks as desired.

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.scrapers.statusinvest import (
    StockListScraper,
    StockExtraInfosScraper,
    StockHistIndicatorsScraper,
    StockHistEarningsScraper,
    StockHistPayoutsScraper,
    StockHistRevenueScraper,
    StockHistMarginsScraper,
    StockHistActivesAndPassivesScraper,
)

### Reading List of Stocks (plus general stock data)

In [2]:
stocks_general_data_df = StockListScraper().dataframe_format

# show most liquid stocks (it also shows only the first X columns)
stocks_general_data_df.sort_values("LIQUIDEZ MEDIA DIARIA", ascending=False, inplace=True)
print(stocks_general_data_df[stocks_general_data_df.columns[0:10]][0:5].to_string(index=False))
print(f"--- Total stock tickers in the list: {stocks_general_data_df.shape[0]} ---")

TICKER  PRECO    DY   P/L  P/VP  P/ATIVOS  MARGEM BRUTA  MARGEM EBIT  MARG. LIQUIDA  P/EBIT
 VALE3  69.50  8.75  6.64  1.66      0.71         40.12        31.07          23.03    4.92
 PETR4  39.96 18.31  3.81  1.35      0.51         51.08        38.69          25.53    2.51
 ITUB4  32.81  3.83 10.10  1.75      0.13         30.03         9.92           8.30    8.46
 BBDC4  15.52  7.17 11.93  0.99      0.09         30.94         3.95           6.18   18.68
 BBAS3  56.97  8.03  4.90  0.99      0.07         32.66        15.94          12.40    3.81
--- Total stock tickers in the list: 620 ---


### Filter out stocks

In [3]:
# by trading liquidity
stocks_general_data_df = stocks_general_data_df[stocks_general_data_df["LIQUIDEZ MEDIA DIARIA"] >= 1_000_000]

# filter out only the most liquid stock from each company
stocks_general_data_df["PREFIX"] = stocks_general_data_df["TICKER"].str.upper().str[:4]
stocks_general_data_df.sort_values("LIQUIDEZ MEDIA DIARIA", ascending=False, inplace=True)
stocks_general_data_df.drop_duplicates("PREFIX", keep="first", inplace=True)
stocks_general_data_df.drop("PREFIX", axis=1, inplace=True)

# show stock tickers count after filter applied
print(f"--- Filtered stock tickers in the list: {stocks_general_data_df.shape[0]} ---")

--- Filtered stock tickers in the list: 219 ---


### Get Historical data from the most liquid stock in the list

In [4]:
stock_ticker = stocks_general_data_df.sort_values("LIQUIDEZ MEDIA DIARIA", ascending=False).index[0]

# show most liquid stock in the list
print(f"--- Most Liquid stock in the list: {stock_ticker} ---")

--- Most Liquid stock in the list: VALE3 ---


In [5]:
extra_infos = StockExtraInfosScraper(stock_ticker).dataframe_format

#show sample of the retrieved data
print(extra_infos)

                           0
Vol Histórica      26.681932
Tag Along             100.0%
Tickers Opções         922.0
Segmento        Novo Mercado
Free Float            98.01%


In [6]:
hist_inds = StockHistIndicatorsScraper(stock_ticker).dataframe_format
hist_inds = hist_inds[sorted(hist_inds.columns)]

#show sample of the retrieved data
print(hist_inds[hist_inds.columns[0:4]])

                                  2008   2009   2010     2011
dy                                 NaN    NaN    NaN   7.3140
p_l                                NaN    NaN    NaN   5.5957
p_vp                               NaN    NaN    NaN   1.4752
p_ebita                            NaN    NaN    NaN      NaN
p_ebit                             NaN    NaN    NaN   4.0409
p_sr                               NaN    NaN    NaN   2.1049
p_ativo                            NaN    NaN    NaN   0.8928
p_capitlgiro                       NaN    NaN    NaN  10.8744
p_ativocirculante                  NaN    NaN    NaN  -1.0748
ev_ebitda                          NaN    NaN    NaN      NaN
ev_ebit                            NaN    NaN    NaN   4.7886
lpa                                NaN    NaN    NaN   7.0501
vpa                                NaN    NaN    NaN  26.7413
peg_Ratio                          NaN    NaN    NaN      NaN
dividaliquida_patrimonioliquido   0.22   0.30   0.27   0.2700
dividali

In [7]:
hist_earnings = StockHistEarningsScraper(stock_ticker).dataframe_format
hist_earnings = hist_earnings[sorted(hist_earnings.columns)]

#show sample of the retrieved data
print(hist_earnings[hist_earnings.columns[0:4]])

              2007      2008      2009      2010
earnings  0.394109  1.089846  1.016622  0.984241


In [8]:
hist_payouts = StockHistPayoutsScraper(stock_ticker).dataframe_format
hist_payouts = hist_payouts[sorted(hist_payouts.columns)]

#show sample of the retrieved data
print(hist_payouts[hist_payouts.columns[0:4]])

        2008  2009  2010       2011
payout   0.0   0.0   0.0  39.549275


In [9]:
hist_revenue = StockHistRevenueScraper(stock_ticker).dataframe_format
hist_revenue = hist_revenue[sorted(hist_revenue.columns)]

#show sample of the retrieved data
print(hist_revenue[hist_revenue.columns[0:4]])

                        2008          2009          2010          2011
receitaLiquida  7.054099e+10  4.849657e+10  8.322501e+10  1.005557e+11
despesas       -3.215562e+10 -2.775039e+10 -3.375607e+10 -4.103345e+10
lucroLiquido    2.127963e+10  1.033695e+10  3.007005e+10  3.782572e+10


In [10]:
hist_margins = StockHistMarginsScraper(stock_ticker).dataframe_format
hist_margins = hist_margins[sorted(hist_margins.columns)]

#show sample of the retrieved data
print(hist_margins[hist_margins.columns[0:4]])

                2008   2009   2010   2011
margemBruta    54.42  42.78  59.44  59.19
margemEbitda    0.00   0.00   0.00   0.00
margemEbit     37.16  27.56  48.59  52.09
margemLiquida  30.78  21.66  36.55  37.21


In [11]:
hist_actives_passives = StockHistActivesAndPassivesScraper(stock_ticker).dataframe_format
hist_actives_passives = hist_actives_passives[sorted(hist_actives_passives.columns)]

#show sample of the retrieved data
print(hist_actives_passives[hist_actives_passives.columns[0:4]])

                              2008          2009          2010          2011
ativoTotal            1.856155e+11  1.777382e+11  2.146621e+11  2.417831e+11
ativoCirculante       5.475374e+10  3.676495e+10  5.426873e+10  4.209522e+10
ativoNaoCirculante    1.308617e+11  1.409732e+11  1.603943e+11  1.996879e+11
passivoTotal          8.461464e+10  7.744296e+10  9.833525e+10  9.509274e+10
passivoCirculante     1.869604e+10  1.747328e+10  2.996756e+10  2.068676e+10
passivoNaoCirculante  0.000000e+00  0.000000e+00  6.695108e+10  7.286767e+10
patrimonioLiquido     1.010008e+11  1.002952e+11  1.163269e+11  1.466904e+11


### Output result into a file

In [12]:
stocks_general_data_df.to_csv("../outputs/statusinvest.csv", sep=";", index=False)